In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch import nn
import numpy as np
import os

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#input = data.to(device)
#model = MyModule(...).to(device)

In [ ]:
device

device(type='cuda', index=0)

# PİPELİNE BAŞLANGIÇ

In [ ]:
# Model
model_yolo = torch.hub.load('ultralytics/yolov5','yolov5x' ,'best_models/best_model_yolo.pt')

In [ ]:
model_main = torchvision.models.mobilenet_v2(pretrained=True).to(device)
#Step 2: Freeze the model parameters
for param in model_main.parameters():
    param.requires_grad = False
model_main.classifier = torch.nn.Linear(in_features=1280, out_features=3).to(device)
model_main.load_state_dict(torch.load('best_models/best_model_main.pt'))
model_main.to(device)
model_main.eval()

In [ ]:
model_KBA = torchvision.models.mobilenet_v2(pretrained=True).to(device)
#Step 2: Freeze the model parameters
for param in model_KBA.parameters():
    param.requires_grad = False
model_KBA.classifier = torch.nn.Linear(in_features=1280, out_features=2).to(device)
model_KBA.load_state_dict(torch.load('best_models/best_model_kba.pt'))
model_KBA.to(device)
model_KBA.eval()

In [ ]:
model_MA = torchvision.models.mobilenet_v2(pretrained=True).to(device)
#Step 2: Freeze the model parameters
for param in model_MA.parameters():
    param.requires_grad = False
model_MA.classifier = torch.nn.Linear(in_features=1280, out_features=2).to(device)
model_MA.load_state_dict(torch.load('best_models/best_model_ma.pt'))
model_MA.to(device)
model_MA.eval()

In [ ]:
model_TA = torchvision.models.mobilenet_v2(pretrained=True).to(device)
#Step 2: Freeze the model parameters
for param in model_TA.parameters():
    param.requires_grad = False
model_TA.classifier = torch.nn.Linear(in_features=1280, out_features=4).to(device)
model_TA.load_state_dict(torch.load('best_models/best_model_ta.pt'))
model_TA.to(device)
model_TA.eval()

In [ ]:
def softmax_(vector_):
  e = np.exp(vector_)
  return e / e.sum()

In [ ]:

main_class_name_dict = {
    0: "KBA",
    1: "MA",
    2: "TA"
}

KBA_class_name_dict = {
    0: "auto",
    1: "car"
}

MA_class_name_dict = {
    0: "lcv",
    1: "motorcycle"
}

TA_class_name_dict = {
    0: "bus",
    1: "multiaxle",
    2: "tractor",
    3: "truck"
}


In [ ]:

from itertools import zip_longest
import time
start = time.time()


file_path = "Vehicle_test_dataset/test10.jpg"
results_yolo = model_yolo(file_path)
crops = results_yolo.crop(save=True)  # cropped detections dictionary,

coords = []
classes = []
pred_proba_ma = ""
pred_proba_ta = ""
pred_proba_kba = ""

for crop in crops:
  class_name = None
  coord = (crop["box"][0].to("cpu").item(),
                              crop["box"][1].to("cpu").item(),
                              crop["box"][2].to("cpu").item(),
                              crop["box"][3].to("cpu").item())
  cropped_arr_arr = crop["im"]

  sf_out = model_main(torch.Tensor(np.expand_dims(np.moveaxis(cropped_arr_arr,2,0).copy(),axis=0)).to(device)).to("cpu").detach().numpy()
  probas = softmax_(sf_out)
  pred_cls_idx, pred_proba = np.argmax(probas),np.max(probas)

  # Apply the classification model to the image
  if pred_proba < 0.80:
    class_name = "vehicle"
  else:
    main_cls_name = main_class_name_dict[pred_cls_idx]
    if main_cls_name == "KBA":
      sf_out_kba = model_KBA(torch.Tensor(np.expand_dims(np.moveaxis(cropped_arr_arr,2,0).copy(),axis=0)).to(device)).to("cpu").detach().numpy()
      probas_kba = softmax_(sf_out_kba)
      pred_cls_idx_kba, pred_proba_kba = np.argmax(probas_kba),np.max(probas_kba)
      if pred_proba_kba < 0.6:
        class_name = "KBA"
      else:
        class_name = KBA_class_name_dict[pred_cls_idx_kba]
        
    elif main_cls_name == "MA":
      sf_out_ma = model_MA(torch.Tensor(np.expand_dims(np.moveaxis(cropped_arr_arr,2,0).copy(),axis=0)).to(device)).to("cpu").detach().numpy()
      probas_ma = softmax_(sf_out_ma)
      pred_cls_idx_ma, pred_proba_ma = np.argmax(probas_ma),np.max(probas_ma)
      if pred_proba_ma < 0.6:
        class_name = "MA"
      else:
        class_name = MA_class_name_dict[pred_cls_idx_ma]
    elif main_cls_name == "TA":
      sf_out_ta = model_TA(torch.Tensor(np.expand_dims(np.moveaxis(cropped_arr_arr,2,0).copy(),axis=0)).to(device)).to("cpu").detach().numpy()
      probas_ta = softmax_(sf_out_ta)
      pred_cls_idx_ta, pred_proba_ta = np.argmax(probas_ta),np.max(probas_ta)
      if pred_proba_ta < 0.6:
        class_name = "TA"
      else:
        class_name = TA_class_name_dict[pred_cls_idx_ta]
  split_path = file_path.rsplit("/", 1)
  with open(str(split_path[1])+".txt", "a") as file:
    file.write(class_name + " " + str(pred_proba) + " " + str(pred_proba_ma) + " " + str(pred_proba_ta) + " " + str(pred_proba_kba) + "\n")

  coords.append(coord) #xyxy
  classes.append(class_name)
  

# Process the result

from PIL import Image, ImageDraw

def visualize_boxes(image, boxes, box_names):
    # Load the image
    im = Image.open(image)

    # Create a drawing object
    draw = ImageDraw.Draw(im)
    

    # Iterate through the boxes and draw them on the image
    for box, name in zip(boxes, box_names):
        x1, y1, x2, y2 = box
        draw.rectangle((x1, y1, x2, y2), outline='red')
        draw.text((x1, y1), name, fill='red')

    # Display the image
    #im.show()
    display(im)
# Example usage

visualize_boxes(file_path, coords, classes)

end = time.time()
print(end - start)


